In [1]:
from tqdm import tqdm
import os

SAVING_DIR = '/home/data/taxonomy'
os.environ["TRANSFORMERS_CACHE"] = SAVING_DIR + "hf_cache/"
os.environ["HF_HOME"] = SAVING_DIR + "hf_cache/"

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import pickle
from transformers import BertModel, BertTokenizer
import torch
import networkx as nx

from gensim.models.fasttext import FastText


device = 'cuda'

In [2]:
import fasttext

In [3]:
ft = fasttext.load_model('../fastText/cc.en.300.bin')

In [4]:
G = nx.read_edgelist('../data/MAG_CS/all.edgelist', create_using=nx.DiGraph, delimiter='\t')

In [ ]:
ft.get_word_vector(list(G.nodes())[2])

In [10]:
nodes = list(G.nodes)
all_embeddings = {}
        
for word in tqdm(nodes):
    all_embeddings[word] = ft.get_word_vector(word)
    

    

  0%|          | 0/29484 [00:00<?, ?it/s]

100%|██████████| 29484/29484 [00:00<00:00, 36471.61it/s]


In [11]:
assert len(all_embeddings.keys()) == len(G.nodes())

In [13]:
with open('../data/MAG_CS/embeddings_fasttext.pickle', 'wb') as f:
    pickle.dump(all_embeddings, f)

In [55]:
all_embeddings[batch[0]]

array([ 1.20797977e-01, -3.81340832e-02,  1.03781968e-01, -1.81674719e-01,
        3.20419401e-01, -3.84520054e-01,  3.51279646e-01,  2.79703051e-01,
       -2.54150450e-01,  7.42966235e-02, -9.56124216e-02, -3.59019518e-01,
        1.04348272e-01,  4.57057178e-01, -7.19119534e-02,  1.37465566e-01,
       -3.09100509e-01,  2.22280413e-01, -1.57886408e-02, -1.31824426e-02,
        2.09146872e-01, -2.34307483e-01,  5.31696007e-02,  2.67984420e-01,
        3.73033553e-01, -5.26787490e-02,  2.41678357e-01,  4.28412892e-02,
       -7.31206685e-03, -2.74351746e-01,  3.21469784e-01,  3.78891043e-02,
       -6.22361079e-02,  3.05091828e-01, -3.73641923e-02,  1.48098916e-02,
       -1.31964400e-01,  2.67978963e-02, -1.51245043e-01,  2.88785577e-01,
       -2.10047618e-01, -5.40464401e-01,  2.20841095e-01, -4.85097691e-02,
        5.08353338e-02, -3.18924814e-01, -2.71863699e-01,  1.10695757e-01,
       -8.22061151e-02, -2.15930834e-01, -3.22926670e-01, -2.22941458e-01,
        1.98351666e-01,  

In [16]:
with torch.no_grad():
    out = model(**tokens)

In [20]:
out.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [25]:
tokens['attention_mask']

tensor([[1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')

In [29]:
masked_outputs = (out['last_hidden_state'] * tokens['attention_mask'].unsqueeze(-1))

In [39]:
embeddings = (masked_outputs.sum(dim=-2) / tokens['attention_mask'].sum(dim=1).unsqueeze(-1))

In [41]:
embeddings.cpu().numpy()

array([[ 0.05684492, -0.4693739 ,  0.19657701, ...,  0.06092722,
        -0.12598057, -0.14159232],
       [-0.00673314, -0.32868305,  0.11917636, ..., -0.1555016 ,
        -0.16298126, -0.02167546],
       [-0.2480003 , -0.5751185 ,  0.14767723, ...,  0.08540212,
        -0.09679956, -0.12009169],
       ...,
       [-0.07648315, -0.24393408, -0.1527339 , ..., -0.27309507,
        -0.1798447 ,  0.10657488],
       [ 0.19035351, -0.00463882,  0.11054821, ...,  0.03654246,
         0.0472547 , -0.03532267],
       [-0.13807954, -0.24475516, -0.16560027, ..., -0.17918664,
        -0.00962719, -0.07841179]], dtype=float32)

In [33]:
batch[-1]

'big bang nucleosynthesis'

In [37]:
tokens

{'input_ids': tensor([[  101, 10147, 12079,   102,     0,     0,     0,     0,     0],
        [  101,  2152,  3177,  7642,  4957,   102,     0,     0,     0],
        [  101, 15453,  5657,   102,     0,     0,     0,     0,     0],
        [  101,  2951,  7790, 10147, 12079,   102,     0,     0,     0],
        [  101, 23760, 13767,   102,     0,     0,     0,     0,     0],
        [  101,  4874, 23760, 13767,  2075,   102,     0,     0,     0],
        [  101, 23760, 18209,  9260,  2291,   102,     0,     0,     0],
        [  101,  5009,  2000,  2592,  4216,   102,     0,     0,     0],
        [  101, 15373,   102,     0,     0,     0,     0,     0,     0],
        [  101,  3609,  5024,   102,     0,     0,     0,     0,     0],
        [  101,  4744,  3409,  2121,   102,     0,     0,     0,     0],
        [  101,  7524,  2707,  2051,   102,     0,     0,     0,     0],
        [  101,  7524,  2203,  2051,   102,     0,     0,     0,     0],
        [  101, 16371, 14321,  2891, 